In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
os.environ['HF_DATASETS_CACHE'] = '/nfs-data2/tungtx2/tmp/huggingface'
os.environ['TRANSFORMERS_CACHE'] = '/nfs-data2/tungtx2/tmp/huggingface'
os.environ['HF_HOME'] = '/nfs-data2/tungtx2/tmp/huggingface'

from unsloth import FastLanguageModel
import torch
from pathlib import Path
import transformers
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from transformers import TextStreamer

/nfs-data2/tungtx2/new_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/nfs-data2/tungtx2/new_env/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-10-19 09:13:58.927031: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-19 09:13:58.941299: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-19 09:13:58.958462: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-19 09:13:58.963498: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-19 09:13:58.976593: I tensorflow/core/platform/cpu_feature_guar

In [3]:
current_dir = Path(os.getcwd())
print(current_dir)

/nfs-data2/tungtx2/Master/seminar2/SOSum_summarization/abstractive


# Finetune LLAMA-3

## Train

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.643 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, 
    bias = "none",    
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
from datasets import load_dataset
from chat_templates import PromptTemplate

ds = load_dataset(
    "json",
    data_files={
        'train': os.path.join(current_dir.parent, 'data', 'train_llm.json'),
        'val': os.path.join(current_dir.parent, 'data', 'val_llm.json'),
        'test': os.path.join(current_dir.parent, 'data', 'test_llm.json')
    }
)

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

template = PromptTemplate(name='Alpaca', keep_keys=['text'], eos_token=EOS_TOKEN)
ds = ds.map(template.process, batched=True)

In [6]:
train_ds = ds['train']
print(train_ds['text'][10])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Summarize the following paragraph, avoid adding any information not presented in the original text.
 To put text, use <code>.text('text')</code> If you want to use <code>.html(SomeValue)</code>, SomeValue should have html tags that can be inside a div it must work too. Just check your script location, as farzad said. Reference: .html and text

### Response:
To put text, use <code>.text('text')</code><|end_of_text|>


In [17]:
# import nltk
# import numpy as np
# from evaluate import load
# import pdb

# metric = load("rouge")

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
#     print(predictions)
#     print(predictions.shape)
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
#     # Rouge expects a newline after each sentence
#     decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
#     decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
#     # Note that other metrics may not have a `use_aggregator` parameter
#     # and thus will return a list, computing a metric for each sentence.
#     result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
#     # Extract a few results
#     result = {key: value * 100 for key, value in result.items()}
    
#     # Add mean generated length
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
#     result["gen_len"] = np.mean(prediction_lens)
    
#     return {k: round(v, 4) for k, v in result.items()}

In [7]:
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = ds['train'],
    eval_dataset = ds['val'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 32,
    # compute_metrics=compute_metrics,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        # sft config
        eval_packing=False,
        dataset_batch_size=12,

        # normal training arguments
        num_train_epochs = 10,
        per_device_train_batch_size = 12,
        per_device_eval_batch_size = 12,
        gradient_accumulation_steps = 1,
        warmup_steps = 120,
        eval_strategy = "steps",
        eval_steps = 1/20,
        save_strategy = "steps",
        save_steps = 1/20,
        save_total_limit = 20,
        load_best_model_at_end = True,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        output_dir = "ckpt/llama3-sosum/exp3_rouge_val",
        seed=42,
        
        metric_for_best_model='loss',
        greater_is_better=False,
    ),
)

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,466 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 12 | Gradient Accumulation steps = 1
\        /    Total batch size = 12 | Total steps = 1,230
 "-____-"     Number of trainable parameters = 83,886,080
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
62,1.074800,1.273632
124,1.372700,1.255028
186,1.162900,1.265088
248,1.006900,1.268864
310,0.991400,1.360322
372,0.498200,1.467966
434,0.562000,1.551718
496,0.214600,1.754602
558,0.223400,1.841665
620,0.090600,2.056012


In [9]:
trainer_stats

TrainOutput(global_step=1230, training_loss=0.46939742697660275, metrics={'train_runtime': 3581.7953, 'train_samples_per_second': 4.093, 'train_steps_per_second': 0.343, 'total_flos': 3.6687440800722125e+17, 'train_loss': 0.46939742697660275, 'epoch': 10.0})

In [9]:
trainer.evaluate()

{'eval_loss': 1.2506458759307861,
 'eval_runtime': 17.9922,
 'eval_samples_per_second': 17.563,
 'eval_steps_per_second': 2.223,
 'epoch': 5.0}

In [10]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, o

## Inference

In [2]:
# load model

max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "ckpt/llama3-sosum/exp3_rouge_val/checkpoint-930", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

# model = trainer.model

model.to('cuda').eval()
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.643 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, o

In [3]:
# load data
from datasets import load_dataset
from chat_templates import PromptTemplate

EOS_TOKEN = tokenizer.eos_token
ds = load_dataset(
    "json",
    data_files={
        'train': 'data/SOSum/train.json',
        'val': 'data/SOSum/val.json',
        'test': 'data/SOSum/test.json',
    }
)

template = PromptTemplate(name='Alpaca', keep_keys=['text', 'instruction', 'input', 'output'], eos_token=EOS_TOKEN)

In [6]:
from transformers import GenerationConfig
import json

save_path = Path('ckpt/llama3-sosum/exp3_rouge_val/model_output/checkpoint_930-test_pred.json')
os.makedirs(save_path.parent, exist_ok=True)

all_result = []
total = len(ds['test'])
for i, item in enumerate(ds['test']):
    gt_text = item['output']
    item['output'] = None
    item = template.process(item)
    enc_inp = tokenizer(item['text'], return_tensors='pt').to('cuda')
    with torch.no_grad():
        output = model.generate(
            **enc_inp, 
            generation_config=GenerationConfig(
                max_new_tokens=384,
                temperature=0.05,
                top_p=0.05,
                eos_token_ids=tokenizer.eos_token,
                early_stopping=True
            )
        )
    decoded_output = tokenizer.batch_decode(output[:, enc_inp.input_ids.shape[1]:-1])[0]
    # eos_index = decoded_output.index('</s>')
    # decoded_output = decoded_output[:eos_index]
    item['output'] = gt_text
    item['pred'] = decoded_output
    all_result.append(item)
    print('Pred: ', decoded_output)
    print('Ground Truth: ', gt_text)
    print(f'Done {i+1}/{total} items\n')
    # if i == 3:
    #     break

with open(save_path, 'w') as f:
    json.dump(all_result, f, ensure_ascii=False)
print(f'Result saved to {save_path}')

Pred:  This is one of the suggested ways of implementing a generic collection in <em>Effective Java; Item 26</em>.
Ground Truth:  You can always do this: <em>However</em> this triggers a warning because it is potentially dangerous, and should be used with caution. Worth mentioning that wherever possible, you'll have a much happier time working with <code>List</code>s rather than arrays if you're using generics.
Done 1/310 items

Pred:  Did you try <code>gitk</code> or <code>gitk --all</code>?
Ground Truth:  Did you try <code>gitk</code> or <code>gitk --all</code> ?
Done 2/310 items

Pred:  Use the <code>collate</code> clause in your query:
Ground Truth:  Use the <code>collate</code> clause in your query: I may not have the syntax exactly right (check BOL), but you can do this to change the collation on-the-fly for the query - you may need to add the clause for each join.
Done 3/310 items

Pred:  here is an example when CROSS APPLY makes a huge difference with performance: Using CROSS A

In [7]:
# load result and evaluate
import evaluate
import json

# metric
rouge_score = evaluate.load("rouge")

pred_data = json.load(open(save_path))
predictions, references = [], []
for item in pred_data:
    predictions.append(item['pred'])
    references.append(item['output'])
    
scores = rouge_score.compute(predictions=predictions, references=references)
scores

{'rouge1': 0.7427219649713959,
 'rouge2': 0.7097012333432113,
 'rougeL': 0.732026965889471,
 'rougeLsum': 0.731539579134558}